# Random forest

In [1]:
from  datetime import datetime, timedelta
# import gc
import numpy as np, pandas as pd
from sklearn.ensemble import RandomForestRegressor
import time	
from sklearn.externals import joblib



/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
prices = pd.read_csv("../datasets/sell_prices.csv", dtype = PRICE_DTYPES)
for col, col_dtype in PRICE_DTYPES.items():
    if col_dtype == "category":
        prices[col] = prices[col].cat.codes.astype("int16")  # 类似于labelencoder，将store_id和item_id数字化表示
        prices[col] -= prices[col].min()  
# prices.head(1)

In [3]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
cal = pd.read_csv("../datasets/calendar.csv", dtype = CAL_DTYPES)
cal["date"] = pd.to_datetime(cal["date"])  #  转化为datetime类型
for col, col_dtype in CAL_DTYPES.items():
    if col_dtype == "category":
        cal[col] = cal[col].cat.codes.astype("int16")  # 将event_name_1/event_type_1/event_name_2/event_type_2weekday数字化表示
        cal[col] -= cal[col].min()  # NaN会转换为-1，该代表将NaN归为0，其他从1开始
# cal.head(1)

In [4]:
h = 28 
max_lags = 57
last_day = 1913  # 历史的最后一天
fday = datetime(2016,4, 25)  # 预测的第一天
fday
nrows_demo=None
# nrows_demo=10000

In [5]:
%time
first_day = 1850
numcols = [f"d_{day}" for day in range(first_day, last_day + 1)]  # 选取历史数据，从first_day到tr_last（包含这一天）d_1200,d_1201
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {numcol: "float32" for numcol in numcols}   # d_1200为float32

dtype.update({col: "category" for col in catcols if col != "id"})  # 除id外，都为category类型
dt = pd.read_csv("../datasets/sales_train_validation.csv", usecols = catcols + numcols, dtype = dtype,nrows=nrows_demo)  # 取catcols和numcols的列(只取了需要的历史天数销量数据)
dt

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


,id,item_id,dept_id,cat_id,store_id,state_id,d_1850,d_1851,d_1852,d_1853,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,4.0,0.0,1.0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,4.0,2.0,1.0,1.0,...,1.0,0.0,5.0,4.0,1.0,0.0,1.0,3.0,7.0,2.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,3.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0.0,0.0,1.0,0.0,...,2.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,1.0,3.0


In [6]:
for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")  # 数字化
            dt[col] -= dt[col].min()
    
    
for day in range(last_day + 1, last_day + 28 +1):  # 遍历预测的每一天
    dt[f"d_{day}"] = np.nan  # 先填充为NaN

dt = pd.melt(dt,
              id_vars = catcols,
              value_vars = [col for col in dt.columns if col.startswith("d_")],
              var_name = "d",
              value_name = "sales")



In [7]:
dt = dt.merge(cal, on= "d", copy = False)
dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [8]:
dt.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1850,0.0,2016-02-21,11604,...,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1851,4.0,2016-02-22,11604,...,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1852,0.0,2016-02-23,11604,...,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1853,1.0,2016-02-24,11604,...,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1854,0.0,2016-02-25,11604,...,2,2016,0,0,0,0,0.0,0.0,0.0,8.26


In [10]:
# FIRST_DAY = 1750

In [12]:
dt["d"]=dt["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1


In [13]:
df = dt.copy()

In [14]:
def create_fea(dt):
    lags = [7, 28]
#     lags = [7]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)  # 新增两列，同一商品前7天的单价和前28天的价格

    wins = [7, 28]
#     wins = [7]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

            # log=7，wins=28表示一个商品当前的参考价为：从当前时间7天前（如1.8号的七天前是1.1号）开始一共向上取28天，求这些天的均值
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",

    }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [15]:
create_fea(df)

In [16]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1850,0.0,2016-02-21,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,7,1,21
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1851,4.0,2016-02-22,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,22
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1852,0.0,2016-02-23,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,23
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1853,1.0,2016-02-24,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,24
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1854,0.0,2016-02-25,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,25


In [17]:
df.dropna(inplace = True)
df.shape

(274410, 31)

In [18]:
df.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [19]:
# cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_type_1", "event_type_2"]
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] 
# useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", 'event_name_1', 'event_name_2']
# useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
useless_cols = ["id", "date", "sales", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
print(train_cols)
X_train = df[train_cols]
y_train = df["sales"]

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd', 'wday',
       'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7',
       'lag_28', 'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28',
       'week', 'quarter', 'mday'],
      dtype='object')


In [20]:
train_cols

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd', 'wday',
       'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7',
       'lag_28', 'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28',
       'week', 'quarter', 'mday'],
      dtype='object')

In [21]:
X_train.head()

,item_id,dept_id,store_id,cat_id,state_id,d,wday,month,year,event_name_1,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
1676950,0,0,0,0,0,1905,1,4,2016,0,...,8.38,1.0,1.0,1.571429,1.000000,1.071429,1.035714,15,2,16
1676951,0,0,0,0,0,1906,2,4,2016,0,...,8.38,2.0,3.0,1.714286,1.428571,1.142857,1.142857,15,2,17
1676952,0,0,0,0,0,1907,3,4,2016,0,...,8.38,0.0,1.0,1.714286,1.571429,1.142857,1.035714,16,2,18
1676953,0,0,0,0,0,1908,4,4,2016,0,...,8.38,0.0,2.0,1.142857,1.714286,1.107143,1.107143,16,2,19
1676954,0,0,0,0,0,1909,5,4,2016,0,...,8.38,0.0,2.0,0.857143,1.857143,1.071429,1.142857,16,2,20


In [23]:

model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

In [24]:
alphas = [1.035, 1.03, 1.025]
# alphas = [1.035]
weights = [1 / len(alphas)] * len(alphas)  # 权重，当前权重值一样
print(weights)

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [25]:
# alphas：利用lgb预测出模型后，乘以alphas才为最终的预测值
# weight：由于有三个alphas，这三个alphas得出的结果各赋予一定的权重

sub = 0.
cols = [f"F{i}" for i in range(1,29)]

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    # te是所有的数据，tst是部分数据
#     te = create_dt(False)  # is_train = False  原始数据
    te = dt

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  # 预测的具体哪一天，比如第1914那天的日期
#         print(tdelta)
        print(icount, day)
#         print((te.date >= day - timedelta(days=max_lags)) & (te.date <= day))

        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()  
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]  # 需要预测的数据
#         print(tst)
        te.loc[te.date == day, "sales"] = alpha * model_rf.predict(tst) # magic multiplier by kyakovlev
#         print(te)
    

#     te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()  # 取出预测的数据  # todo 此处 copy 意义？
    te_sub = te.loc[te.date >= fday, ["id", "sales"]]  # 取出预测的数据  # todo 此处 copy 意义？
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  # 转变成submission的形式（只是index只有每种商品的预测值，没有分位数和汇总的一些）
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
#     te_sub.to_csv(f"submission_rf_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)



0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1.035 0.3333333333333333
1 2016-04-25 00:00:00
1 2016-04-26 00:00:00
1 2016-04-27 00:00:00
1 2016-04-28 00:00:00
1 2016-04-29 00:00:00
1 2016-04-30 00:00:00
1 2016-05-01 00:00:00
1 2016-05-02 00:00:00
1 2016-05-03 00:00:00
1 2016-05-04 00:00:00
1 2016-05-05 00:00:00
1 2016-05-06 00:00:00
1 2016-05-07 00:00:00
1 2016-05-08 00:00:00
1 2016-05-09 00:00:00
1 2016-05-10 00:00:00
1 201

In [26]:
#----

In [27]:
print(sub.shape)
print(sub.columns)
sub.head()

(30490, 29)
Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object', name='F')


F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,2.403333,1.030000,1.030000,0.343333,1.030000,0.343333,5.493333,4.463333,1.716667,...,0.343333,1.030000,0.000000,1.373333,1.030000,1.030000,0.343333,1.030000,1.030000,0.343333
1,FOODS_1_001_CA_2_validation,0.343333,0.343333,0.343333,1.716667,0.343333,0.343333,1.716667,1.373333,0.343333,...,0.343333,0.343333,1.030000,0.686667,0.686667,1.030000,1.030000,0.686667,0.686667,1.030000
2,FOODS_1_001_CA_3_validation,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.000000,0.686667,0.343333,...,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667
3,FOODS_1_001_CA_4_validation,0.343333,0.686667,0.686667,0.686667,0.686667,0.686667,0.000000,0.686667,0.686667,...,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667
4,FOODS_1_001_TX_1_validation,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.000000,0.686667,0.686667,...,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333


In [28]:

sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
# sub.to_csv("submission_rf.csv",index=False)

In [29]:
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,2.403333,1.030000,1.030000,0.343333,1.030000,0.343333,5.493333,4.463333,1.716667,...,0.343333,1.030000,0.000000,1.373333,1.030000,1.030000,0.343333,1.030000,1.030000,0.343333
1,FOODS_1_001_CA_2_validation,0.343333,0.343333,0.343333,1.716667,0.343333,0.343333,1.716667,1.373333,0.343333,...,0.343333,0.343333,1.030000,0.686667,0.686667,1.030000,1.030000,0.686667,0.686667,1.030000
2,FOODS_1_001_CA_3_validation,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.000000,0.686667,0.343333,...,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667
3,FOODS_1_001_CA_4_validation,0.343333,0.686667,0.686667,0.686667,0.686667,0.686667,0.000000,0.686667,0.686667,...,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667
4,FOODS_1_001_TX_1_validation,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.000000,0.686667,0.686667,...,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333,0.343333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.686667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.343333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
sub.to_csv("submission_rf.csv",index=False)